<a href="https://colab.research.google.com/github/rakshith3101/IG_gateway_task/blob/main/M2_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/anger-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/fear-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/joy-ratings-0to1.train.txt http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/sadness-ratings-0to1.train.txt
!wget http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/anger-ratings-0to1.test.target.txt http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/fear-ratings-0to1.test.target.txt http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/joy-ratings-0to1.test.target.txt  http://saifmohammad.com/WebDocs/EmoInt%20Test%20Data/sadness-ratings-0to1.test.target.txt

--2024-03-10 10:40:16--  http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/anger-ratings-0to1.train.txt
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95686 (93K) [text/plain]
Saving to: ‘anger-ratings-0to1.train.txt’

anger-ratings-0to1. 100%[===================>]  93.44K  --.-KB/s    in 0.1s    

2024-03-10 10:40:17 (680 KB/s) - ‘anger-ratings-0to1.train.txt’ saved [95686/95686]

--2024-03-10 10:40:17--  http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/fear-ratings-0to1.train.txt
Reusing existing connection to saifmohammad.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 133186 (130K) [text/plain]
Saving to: ‘fear-ratings-0to1.train.txt’

fear-ratings-0to1.t 100%[===================>] 130.06K  --.-KB/s    in 0.05s   

2024-03-10 10:40:17 (2.78 MB/s) - ‘fear-ratings-0to1.train.txt’ saved [133186/133186]

--202

In [2]:
#train_data
import io
file_content=['/content/anger-ratings-0to1.train.txt','/content/fear-ratings-0to1.train.txt','/content/joy-ratings-0to1.train.txt','/content/sadness-ratings-0to1.train.txt']
output_file='/content/training_data.txt'
with open(output_file, 'w') as output:
    # Write the header line to the output file
    output.write("id\ttweet\tlabel\tintensity\n")

    # Loop through each input file
    for file_name in file_content:
        # Open the input file in read mode
        with open(file_name, 'r') as input_file:
            # Skip the header line in the input file
            next(input_file)

            # Copy the content of the input file to the output file
            output.write(input_file.read())

print(f"Combined data from {', '.join(file_content)} into {output_file}.")

Combined data from /content/anger-ratings-0to1.train.txt, /content/fear-ratings-0to1.train.txt, /content/joy-ratings-0to1.train.txt, /content/sadness-ratings-0to1.train.txt into /content/training_data.txt.


In [3]:
#test_data
import io
file_content=['/content/anger-ratings-0to1.test.target.txt','/content/fear-ratings-0to1.test.target.txt','/content/joy-ratings-0to1.test.target.txt','/content/sadness-ratings-0to1.test.target.txt']
output_file='/content/test_data.txt'
with open(output_file, 'w') as output:
    # Write the header line to the output file
    output.write("id\ttweet\tlabel\tintensity\n")

    # Loop through each input file
    for file_name in file_content:
        # Open the input file in read mode
        with open(file_name, 'r') as input_file:
            # Skip the header line in the input file
            next(input_file)

            # Copy the content of the input file to the output file
            output.write(input_file.read())

print(f"Combined data from {', '.join(file_content)} into {output_file}.")

Combined data from /content/anger-ratings-0to1.test.target.txt, /content/fear-ratings-0to1.test.target.txt, /content/joy-ratings-0to1.test.target.txt, /content/sadness-ratings-0to1.test.target.txt into /content/test_data.txt.


In [4]:
import pandas as pd
train_data = pd.read_csv('/content/training_data.txt', delimiter='\t')
test_data=pd.read_csv('/content/test_data.txt',delimiter='\t')

In [5]:
label_mapping={'anger':0,'fear':1,'joy':2,'sadness':3}
train_data['label']=train_data['label'].map(label_mapping)
train_data.head()
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3609 entries, 0 to 3608
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         3609 non-null   int64  
 1   tweet      3609 non-null   object 
 2   label      3609 non-null   int64  
 3   intensity  3609 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 112.9+ KB


In [6]:
test_data=test_data.drop(columns=['intensity'])

In [7]:
!pip install neattext
import neattext.functions as nfx
train_data['clean_text']= train_data['tweet'].apply(nfx.remove_userhandles)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_hashtags)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_stopwords)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_emojis)
train_data['clean_text']=train_data['clean_text'].apply(nfx.remove_punctuations)
train_data['clean_text'] = train_data['clean_text'].apply(lambda x: x.lower())
train_data.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.6 MB/s eta 0:00:00


,id,tweet,label,intensity,clean_text
0,10001,So my Indian Uber driver just called someone t...,0,0.896,indian uber driver called n word moving vehicl...
1,10002,@DPD_UK I asked for my parcel to be delivered ...,0,0.896,asked parcel delivered pick store address
2,10003,so ef whichever butt wipe pulled the fire alar...,0,0.896,ef whichever butt wipe pulled fire alarm davis...
3,10004,Don't join @BTCare they put the phone down on ...,0,0.896,join phone you talk rude taking money acc will...
4,10005,My blood is boiling,0,0.875,blood boiling


In [8]:
test_data['clean_text']=test_data['tweet'].apply(nfx.remove_userhandles)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_hashtags)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_stopwords)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_emojis)
test_data['clean_text']=test_data['clean_text'].apply(nfx.remove_punctuations)
test_data['clean_text']=test_data['clean_text'].apply(lambda x: x.lower())


In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_data['encoded_labels'] = label_encoder.fit_transform(train_data['label'])
test_data['encoded_labels']=label_encoder.fit_transform(test_data['label'])

In [10]:
x_train=train_data.clean_text
y_train=train_data.label
x_test=test_data.clean_text
y_test=test_data.label

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
top_words = 6000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(x_train)
list_tokenized_train = tokenizer.texts_to_sequences(x_train)

max_review_length = 130
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)



In [19]:
y_train.classes

AttributeError: 'Series' object has no attribute 'classes'

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate
from sklearn.preprocessing import StandardScaler
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 130, 32)           192032    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245333 (958.33 KB)
Trainable params: 245333 (958.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.fit(X_train,y_train, epochs=25, batch_size=64, validation_split=0.2)

Epoch 1/25
46/46 [==============================] - 11s 181ms/step - loss: -0.0296 - accuracy: 0.3956 - val_loss: -19.1944 - val_accuracy: 0.0000e+00
Epoch 2/25
46/46 [==============================] - 8s 181ms/step - loss: -0.3767 - accuracy: 0.3970 - val_loss: -26.3393 - val_accuracy: 0.0000e+00
Epoch 3/25
46/46 [==============================] - 9s 205ms/step - loss: -0.4913 - accuracy: 0.3970 - val_loss: -30.5975 - val_accuracy: 0.0000e+00
Epoch 4/25
46/46 [==============================] - 7s 156ms/step - loss: -0.6151 - accuracy: 0.3970 - val_loss: -30.9256 - val_accuracy: 0.0000e+00
Epoch 5/25
46/46 [==============================] - 8s 180ms/step - loss: -1.5040 - accuracy: 0.3970 - val_loss: -20.9036 - val_accuracy: 0.0000e+00
Epoch 6/25
46/46 [==============================] - 8s 179ms/step - loss: -4.1669 - accuracy: 0.4396 - val_loss: -17.1192 - val_accuracy: 0.0000e+00
Epoch 7/25
46/46 [==============================] - 7s 157ms/step - loss: -6.6897 - accuracy: 0.5265 - va